In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
from tqdm.auto import tqdm
from dataset import get_labelled, get_unlabelled, _preload
from Commit import CommitFactory
from Model import CommitDiffModelFactory

In [2]:
CONTEXT_SIZE = 16
BAG_SIZE = 512
OUTPUT_SIZE = 128

In [3]:
Commit = CommitFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)
CommitDiffModel = CommitDiffModelFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE, OUTPUT_SIZE=OUTPUT_SIZE)

In [4]:
_preload()

Loading Commit lookup table


  0%|          | 0/7 [00:00<?, ?it/s]

Loading file ../data/commit_lookups/supervised_commit_data_lookup0-1000.pickle
Appending pickle of length: 184 , new dict length: 184
Loading file ../data/commit_lookups/commit_data_lookup77500-80000.pickle
Appending pickle of length: 2500 , new dict length: 2684
Loading file ../data/commit_lookups/commit_data_lookup75000-77500.pickle
Appending pickle of length: 2499 , new dict length: 5183
Loading file ../data/commit_lookups/commit_data_lookup12500-15000.pickle
Appending pickle of length: 2497 , new dict length: 7579
Loading file ../data/commit_lookups/commit_data_lookup72500-75000.pickle
Appending pickle of length: 2500 , new dict length: 10023
Loading file ../data/commit_lookups/commit_data_lookup10000-12500.pickle
Appending pickle of length: 2496 , new dict length: 12372
Loading file ../data/commit_lookups/commit_data_lookup70000-72500.pickle
Appending pickle of length: 2497 , new dict length: 14777


In [5]:
X_train, X_test, y_train, y_test = get_labelled(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)

Generating Positive X_train:   0%|          | 0/97 [00:00<?, ?it/s]

Generating Positive y_train:   0%|          | 0/97 [00:00<?, ?it/s]

Generating Negative X_train:   0%|          | 0/70 [00:00<?, ?it/s]

Generating Negative y_train:   0%|          | 0/70 [00:00<?, ?it/s]

In [6]:
X_train_unsupervised = get_unlabelled(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)

Generating Unsupervised X_train:   0%|          | 0/14777 [00:00<?, ?it/s]

In [7]:
print("Train set size", len(y_train))
print("Train set split", np.sum(y_train)/len(y_train))
print("Test set size", len(y_test))
print("Test set split", np.sum(y_test)/len(y_test))
print("Unsupervised Train Size", len(X_train_unsupervised))

Train set size 132
Train set split 0.5833333333333334
Test set size 33
Test set split 0.5454545454545454
Unsupervised Train Size 14777


In [ ]:
for encoder in tqdm([0,1,7,8,9,10,11,6,2,3,4,5]):
    model = CommitDiffModel()
    model.initialize(encoder)
    model.fit_siam(np.array(X_train_unsupervised), epochs=4, batch_size=256, verbose=1)
    model.fit_binary_classification(X_train, np.array(y_train), epochs=8, batch_size=32, verbose=1)
    score = model.evaluate_binary_classification(X_test, np.array(y_test), verbose=0)
    print("Enocder:", encoder)
    print("Score:", score)

  0%|          | 0/12 [00:00<?, ?it/s]

Epoch 1/4
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
58/58 [==============================] - 11s 125ms/step - loss: -0.4630
Epoch 2/4
58/58 [==============================] - 7s 116ms/step - loss: -0.6674
Epoch 3/4
58/58 [==============================] - 6s 108ms/step - loss: -0.6965
Epoch 4/4
58/58 [==============================] - 7s 113ms/step - loss: -0.7096
Epoch 1/8
5/5 [==============================] - 4s 41ms/step - loss: 0.7895
Epoch 2/8
5/5 [==============================] - 0s 34ms/step - loss: 0.7752
Epoch 3/8
5/5 [==============================] - 0s 38ms/step - loss: 0.7390
Epoch 4/8
5/5 [==============================] - 0s 36ms/step - loss: 0.7136
Epoch 5/8
5/5 [==============================] - 0s 34ms/step - loss: 0.7115
Epoch 6/8
5/5 [==============================] - 0s 35ms/step - loss: 0.6848
Epoch 7/8
5/